# Problem 1

In [1]:
%config InlineBackend.figure_format = 'svg'
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
from lestack import *
import mpmath
phi = scipy.stats.norm.pdf
invPhi = scipy.stats.norm.ppf;

In [2]:
# My functions:
from solrel import biMorgen, siSet, nataf, FORM

## Set up

#### Distributions

In [3]:
mean1, std1 = 1200., 300.
mean2, std2 = 12, 4.8
rho = 0.5

Rxx  = np.array([[1, rho],[rho,1]])
Mx = np.array([[mean1],[mean2]])
Dx = np.array([[std1,0.0],[0.0,std2]]);

In [4]:
# Compute parameters
lam1 = np.log(mean1)-0.5*np.log(1+(std1/mean1)**2)
zeta = np.sqrt(np.log(1+(std1/mean1)**2))

lam2 = mean2/std2**2
k = (mean2/std2)**2;

In [5]:
# Define marginal distributions
s = np.sqrt(np.log(std1**2/mean1**2+1))
mu = mean1**2/np.sqrt(std1**2+mean1**2)
X1 = scipy.stats.lognorm(s=s, scale=mu);
X2 = scipy.stats.gamma(a=k, scale=1/lam2);
print(X1.mean(),' ',X1.std())
print(X2.mean(),' ',X2.std())

1200.0000000000002   300.00000000000006
12.0   4.8


In [6]:
# Define multivariate distribution
def C(delj,deli,rho): 
    return 1.001 + 0.033*rho + 0.004*deli - 0.016*delj + 0.002*rho**2 + 0.223*deli**2 + 0.130*delj**2 + 0.029*delj*deli - 0.104*rho*deli - 0.119*rho*delj
XYii = nataf([X1,X2], Rxx, C);

#### Limit State

In [7]:
def func_x(x):
    theta = 0.5
    x1,x2 = x[0],x[1]
    return x1**theta - x2

def grad_x(x):
    theta = 0.5
    return np.array ([theta*x[0]**(theta-1),-1])

#### FORM

In [8]:
form = FORM(XYii,func_x,grad_x)
form.run()
x = form.design_point_x
u = form.design_point_u
z = XYii.x_to_z(x)
alpha = form.alpha
beta = form.beta
pf = form.pf
form.print_analysis()



FORM Results:
u*:    [0.26759251 3.80124488]
x*:    [1243.45781056   35.26269682]
alpha: [0.07023099 0.99753076]
beta:  3.810651968512857
pf1:   6.930038946618659e-05


## a)

In [9]:
form.print_analysis(form=False,sensitivity=True)



Sensitivity:
alpha: [0.0702 0.9975]
gamma: [-0.4126  0.9109]


$Imp(U_2) > Imp(U_1)$\
$Imp(X_2) > Imp(X_1)$\

$\hat{X}_1$ is a capacity variable
$\hat{X}_2, U_1$, and $U_2$ are demand variables

## b)

### Sensitivity of $\beta$ and $p_f$ w.r.t. $\theta$

In [10]:
grad_theta_beta = 30.46 # Hand calcs

30.46

In [11]:
grad_theta_pf = -phi(beta)*grad_theta_beta

-0.008539452371941867

### Sensitivity of $\beta$ and $p_f$ w.r.t. $\lambda, \zeta, \lambda_2, k$

In [12]:
J_u_tf = np.zeros((2,4))
L0i = XYii.invL0
J_u_tf[0,0] = L0i[0,0] * (-1/zeta)
J_u_tf[0,1] = L0i[0,0] * (-np.log(x[0])-lam1)/zeta**2
J_u_tf[1,2] = L0i[1,1] * 1/phi(z[1])*1/scipy.special.gamma(k)*(-lam2**(k-1)*np.exp(-x[1]))

In [13]:
f = lambda t: np.exp(-t)*t**(k-1)*np.log(t)
dG_dk = scipy.integrate.quad(f,0,np.inf)[0] # Derivative of gamma func w.r.t k

T = lambda k,lam2: mpmath.meijerg(a_s=[[],[0,0]], b_s=[[k-1,-1,-1],[]], z=lam2)
dGi_dk = np.log(lam2)*scipy.special.gammainc(k,lam2)+lam2*T(k,lam2) # Derivative of incomplete gamma func w.r.t k

J_u_tf[1,3] = L0i[1,1] * (1/scipy.special.gamma(k)**2*(scipy.special.gamma(k)*dGi_dk-dG_dk*scipy.special.gammainc(k,lam2)))
grad_tf_beta = alpha@J_u_tf

array([-2.85235973e-01, -1.64331889e+01, -8.04398075e-17,  2.79287622e+00])

In [14]:
grad_tf_pf = -phi(beta)*grad_tf_beta

array([ 7.99658242e-05,  4.60703985e-03,  2.25512772e-20, -7.82982057e-04])

## c)

### Sensitivities of $\beta,p_f$ w.r.t. means and std

In [15]:
J_t_tf = np.zeros((4,4))
del1 = (std1/mean1)
J_t_tf[0,0] = (1+2*del1**2)/(mean1*(1+del1**2))
J_t_tf[0,1] = -del1/(mean1*(1+del1**2))
J_t_tf[1,0] = -del1**2/(zeta*mean1*(1+del1**2))
J_t_tf[1,1] = -del1/(zeta*mean1*(1+del1**2))
J_t_tf[2,2] = std2**(-2)                    # dlam2 wrt mean2
J_t_tf[2,3] = -2*mean2/(std2**3)            # dlam2 wrt std2
J_t_tf[3,2] = 2*mean2/std2**2               # dk wrt mean2
J_t_tf[3,3] = -2*(mean2/std2)**2/std2       # dk wrt std2
grad_tfprime_beta = grad_tf_beta@J_t_tf;

In [16]:
grad_M_beta = grad_tfprime_beta[[0,2]]

array([0.00301997, 2.90924606])

In [17]:
grad_D_beta = grad_tfprime_beta[[1,3]]

array([ 0.01314254, -7.27311516])

In [18]:
grad_M_pf = -phi(beta)*grad_M_beta

array([-8.46648750e-07, -8.15606309e-04])

In [19]:
grad_D_pf = -phi(beta)*grad_D_beta

array([-3.68450689e-06,  2.03901577e-03])

### Sensitivity of $\beta$ and $p_f$ w.r.t. mean and c.o.v

In [20]:
J_std_cov = np.zeros((4,4))
J_std_cov[0,0] = 1.0
J_std_cov[0,1] = 0.0
J_std_cov[1,0] = std1/mean1
J_std_cov[1,1] = mean1

J_std_cov[2,2] = 1.0
J_std_cov[2,3] = 0.0
J_std_cov[3,2] = std2/mean2
J_std_cov[3,3] = mean2
grad_tfpprime_beta = grad_tfprime_beta@J_std_cov;

In [21]:
grad_Mcov_beta = grad_tfpprime_beta[[0,2]]

array([6.30560820e-03, 1.53425555e-16])

In [22]:
grad_cov_beta = grad_tfpprime_beta[[1,3]]

array([ 15.77104598, -87.27738186])

In [23]:
grad_Mcov_pf = -phi(beta)*grad_Mcov_beta

array([-1.76777547e-06, -4.30128109e-20])

In [24]:
grad_cov_pf = -phi(beta)*grad_cov_beta

array([-0.00442141,  0.02446819])

## d)

### $\delta$

In [25]:
grad_M_beta@Dx

array([ 0.90599209, 13.9643811 ])

$Imp(\mu_2) > Imp(\mu_1)$

### $\eta$

In [26]:
grad_D_beta@Dx

array([  3.94276149, -34.91095274])

$Imp(\sigma_2) > Imp(\sigma_1)$

## e)

In [27]:
d_std1 = 310 - std1 
d_mean2 = 12.5 - mean2
d_tf = np.array([0.0, d_std1, d_mean2, 0.0]).T

array([ 0. , 10. ,  0.5,  0. ])

In [28]:
new_beta = beta + d_tf.T@grad_tfprime_beta

5.3967003826848305